In [1]:
"""
rebound_path_checker.py

用途：
- 根据用户描述的规则对给定的截断差分路径 (solution) 进行自由度 / 概率计数检查
- 输出 inbound/outbound 的 DoF 估算与几个概率类计数项
- 生成一个带组 id 的差分路径矩阵，标注哪些块是通过直接异或“相等/传递”关系绑定在一起

假设：
- solution: rounds x 12 的 0/1 矩阵（或列表的列表）
- 每个活跃字节在计数上视为 1 个“自由度单元”（你可按需把它换成 8 bits）
- 轮之间的依赖（哪个块由哪些块通过 XOR/AES 得到）由用户以映射表形式提供

如何使用：
1. 将你的 solution 传入 `solution` 变量（或从文件加载）
2. 填写 `dependencies` 映射：dependencies[(to_round, to_block)] = list of sources，
   其中每个 source 是 (from_round, from_block, via) ，via 为 'xor' 或 'aes'
   （若一个目标由多个源通过 xor 合并，则会有多个 source，且 via='xor'）
3. 设置 inbound_start_round, inbound_end_round, inbound_start_blocks, inbound_end_blocks
   outbound 相应设置
4. 运行脚本，查看输出计数与带组 id 的路径

注意：本脚本实现了你描述的计数策略的直接映射（尽量忠实于你的规则）。
"""

from collections import defaultdict, deque

# --------------------------
# 用户需要提供 / 修改的部分
# --------------------------

# 示例 solution：10 轮 × 12 block（这里只是示例；真实用你的 solution 代替）
# 每个条目 0/1，1 表示该 block 在该轮是活跃的
# 你应该用你的实际 solution 赋值给这个变量
solution = [
    # 12 blocks per round
    [0,1,0,0,0,1, 0,0,0,0,0,0],  # round 0
    [0,1,0,0,0,0, 0,1,0,0,0,0],  # round 1
    [0,0,0,1,0,0, 0,0,0,0,1,0],  # ...
    [0,0,0,0,0,0, 0,0,1,0,0,0],
    [0,0,0,0,1,0, 0,0,0,0,0,1],
    [0,0,1,0,0,0, 1,0,0,0,0,0],
    [0,0,0,0,0,1, 0,0,0,1,0,0],
    [1,0,0,0,0,0, 0,0,0,0,0,0],
    [0,0,0,0,1,0, 0,0,0,0,0,0],
    [0,0,0,0,0,0, 0,0,0,0,0,0],
]

rounds = len(solution)
num_blocks = 12

# Inbound / Outbound 信息（示例）
inbound_begin_round = 1   # inbound 起点轮号（包含）
inbound_end_round = 3     # inbound 终点轮号（包含）
# inbound 两端实际活跃的 blocks 列表（可从 solution 中直接读，也可显式给出）
# 这里举例：start layer = round inbound_begin_round 的活跃块索引
inbound_start_blocks = [i for i, v in enumerate(solution[inbound_begin_round]) if v==1]
inbound_end_blocks = [i for i, v in enumerate(solution[inbound_end_round]) if v==1]

outbound_begin_round = 6  # outbound 起点轮号（包含）
outbound_end_round = 8    # outbound 终点轮号（包含）
outbound_start_blocks = [i for i, v in enumerate(solution[outbound_begin_round]) if v==1]
outbound_end_blocks = [i for i, v in enumerate(solution[outbound_end_round]) if v==1]

# --------------------------
# 依赖映射：必须由用户提供（示例）
# dependencies[(to_round, to_block)] = [ (from_round, from_block, via), ... ]
# via: 'xor' 或 'aes' （可扩展）
#
# 解释：表示 (to_round,to_block) 的值是由 listed sources 经过 XOR / AES 等得到的。
# 如果只有一个 source 且 via == 'aes'，表示经过 S-box（非线性）从该 source 传来。
# 如果多个 source 并且 via == 'xor'，表示目标为这些 source 的 XOR。
#
# 你需要把这个表填成与你的轮函数精确对应的结构。
# 下面给出一个**示例**映射（任意示例，仅为演示如何使用函数）。
# --------------------------
dependencies = dict()

# 示例构造：假设每个 round t 的 block b 等于：
#    AES( round t-1, block b ) XOR AES( round t-1, block (b+1)%12 )
# 较简单的示例映射（真实请替换）
for t in range(1, rounds):
    for b in range(num_blocks):
        # 目标 (t,b) 由两个 AES 源异或得到
        dependencies[(t,b)] = [
            (t-1, b, 'aes'),
            (t-1, (b+1) % num_blocks, 'aes'),
        ]
# 边界 round=0 未定义 sources，为输入/明文层
# 真实场景中请根据你的轮函数构造 dependencies

# --------------------------
# 计数器与检查器实现（不需要修改）
# --------------------------

def propagate_and_count(solution, dependencies,
                        inbound_begin_round, inbound_end_round,
                        inbound_start_blocks, inbound_end_blocks,
                        outbound_begin_round, outbound_end_round,
                        outbound_start_blocks, outbound_end_blocks):
    """
    按你给的规则，从 inbound 两端向内推进，计算 DoF_In, Prob_In_xor, Prob_In_MC
    同时从 outbound 两端向外推进，计算 DoF_Out, Prob_Out_xor, Prob_Out_MC

    返回：
      {
        'DoF_In': int,
        'Prob_In_xor': int,
        'Prob_In_MC': int,
        'DoF_Out': int,
        'Prob_Out_xor': int,
        'Prob_Out_MC': int,
        'annotated_path': matrix rounds x num_blocks with group ids (int, 0 表示 inactive)
      }
    """
    R = len(solution)
    B = num_blocks

    # 标记已计入自由度（避免重复计数）：按 (round,block) 存布尔
    counted_dof = set()

    # 记录活跃状态（bool）从 solution 读取（防止后面修改原数组）
    active = {(r,b): bool(solution[r][b]) for r in range(R) for b in range(B)}

    # 输出计数器
    DoF_In = 0
    Prob_In_xor = 0
    Prob_In_MC = 0

    DoF_Out = 0
    Prob_Out_xor = 0
    Prob_Out_MC = 0

    # helper: count a DOF if not counted yet
    def count_dof(r,b,where='in'):
        nonlocal DoF_In, DoF_Out
        if (r,b) in counted_dof:
            return False
        counted_dof.add((r,b))
        if where == 'in':
            DoF_In += 1
        else:
            DoF_Out += 1
        return True

    # 从 inbound 两端开始：将 start/end 的活跃字节都作为初始 DOF（你说的 “DoF In +1”）
    for b in inbound_start_blocks:
        if active.get((inbound_begin_round,b), False):
            count_dof(inbound_begin_round,b,'in')
    for b in inbound_end_blocks:
        if active.get((inbound_end_round,b), False):
            count_dof(inbound_end_round,b,'in')

    # 从 outbound 两端开始
    for b in outbound_start_blocks:
        if active.get((outbound_begin_round,b), False):
            count_dof(outbound_begin_round,b,'out')
    for b in outbound_end_blocks:
        if active.get((outbound_end_round,b), False):
            count_dof(outbound_end_round,b,'out')

    # 定义推进方向函数：给定当前 round -> 计算下一层的活跃与计数变化
    # 基本规则根据你描述的方法来实现（尽量忠实）：
    # - 如果目标由 XOR 合成：
    #   - 若源中有(active+active -> result inactive) 的情况，Prob_xor +=1
    #   - XOR 合成本身不引入新的 DoF（除非其中一侧通过 AES 且那个侧的活跃字节未被计入，则计入 DoF）
    # - 如果目标通过 AES（非线性）从一个源传来：
    #   - 若目标是活跃并且源端有活跃且源端未计数 -> 计入 DoF (+1)
    # - 对 MC（MixColumns / 多输出情形）：当某一步产生了 j 个活跃字节时，Prob_MC += j
    #
    # 注：这里将 "MC 之后出现 j 个活跃字节" 理解为当目标由一个 source 经过一个线性多输出映射(例如 MDS)得到多个活跃输出
    # 我实现时如果 dependencies 列表里同一个 from 出现多次或目标同源多个位置，会把它视作 MC 产生活跃计数。
    #

    # 为了推进，我们按从 inbound 较外侧向中心逐轮推进（从 inbound_begin_round+1 到 inbound_end_round-1）
    # 同理 outbound 方向从 outbound_begin_round-1 向外推进到 outbound_end_round+1
    # 这里我们做简单的 BFS/迭代直到稳定（不再产生新的计数）
    changed = True
    # 为区分 in/out 计数使用标记：我们在 counted_dof 里只记录已计入 DOF 的位置
    # 对 Prob_* 的计数会在每次发现 xor/MC 的特定模式时增加

    # 为了稳定性，我们做一个固定次循环或直到 no-change
    iter_limit = R * B * 4
    it = 0
    while it < iter_limit:
        it += 1
        changed = False

        # 遍历所有目标位置，根据其 sources 分析
        for (t,b), sources in dependencies.items():
            # sources: list[(r_from, b_from, via)]
            # 如果没有定义 source (例如 t==0) 跳过
            if not sources:
                continue

            # 计算源活跃数量与每个源的 via 类型
            src_active = [(s[0], s[1], s[2], active.get((s[0], s[1]), False)) for s in sources]
            active_count = sum(1 for s in src_active if s[3])
            target_act = active.get((t,b), False)

            # 情形1：XOR 合并（多个源且 via 为 'xor'）
            if len(sources) >= 2 and all(s[2]=='xor' or s[2]=='aes' for s in sources):
                # 我们将 XOR 情形与 AES 混合情况都覆盖：若多个源存在，认定逻辑是各源按其 through 被合并（你的实际轮函数决定）
                # 当发生 active+active => non-active（即源个数为2，且两个源都活跃但目标不活跃）:
                if active_count >= 2 and not target_act:
                    # 按你说的规则，Prob_In_xor / Prob_Out_xor 增加 1
                    # 我们无法直接区分这是 inbound 方向还是 outbound，尝试靠源/目标是否处于 inbound/outbound 区间判断
                    # 更保守：增加两个统计里的相应项（也可以只增加 inbound 或 outbound，视需要）
                    Prob_In_xor += 1
                    Prob_Out_xor += 1
                    changed = True

                # 如果 XOR 合并后目标活跃，但这些活跃是由线性组合产生，不直接增加 DoF，
                # 但是如果某个源通过 AES（非线性）传来且该源的活跃字节尚未计入 DOF，则计入
                for (fr, fb, via, isact) in src_active:
                    if isact and via == 'aes' and (fr,fb) not in counted_dof:
                        # 增加 DOF（把这侧计作一个需要选择的自由变量）
                        # 决定把它们归为 inbound 还是 outbound：用它们在轮序上的位置靠近 inbound 区间来判断
                        # 如果 fr 在 inbound 区间内则记入 In，否则记入 Out；否则默认记入 In（保守策略）
                        where = 'in' if inbound_begin_round <= fr <= inbound_end_round else 'out'
                        count_dof(fr, fb, where)
                        changed = True

            # 情形2：单源通过 AES 到多目标（类似 MixColumns/MC 的逆向：一个 source 生成 j 个活跃）
            # 在 dependencies 表里若一个 from 出现在多个不同目标的 source 列表中，这意味着 MC-like 扩散
            # 因而我们扫描是否存在这样一个 from 在多个不同 (t',b') 的 sources 中
            # 我们实现简单检测：若当前 (t,b) 的某个 source (fr,fb,'aes')，且该 source 在同一轮的其他目标里也出现，
            # 并且 those targets 有活跃数 j，则 Prob_*_MC += j
            for (fr, fb, via, isact) in src_active:
                if via == 'aes':
                    # find how many targets this (fr,fb) maps to (in dependencies)
                    mapped_targets = [(tt,bb) for (tt,bb), srcs in dependencies.items()
                                      if any(s[0]==fr and s[1]==fb and s[2]=='aes' for s in srcs)]
                    # count how many of these targets are active now
                    j = sum(1 for (tt,bb) in mapped_targets if active.get((tt,bb), False))
                    if j > 0:
                        Prob_In_MC += j
                        Prob_Out_MC += j
                        changed = True

            # 额外规则：如果目标通过 AES 从某个源传来并且目标活跃，而源端有活跃但未计数 -> 增加 DOF
            for (fr, fb, via, isact) in src_active:
                if via == 'aes' and target_act and isact and (fr,fb) not in counted_dof:
                    where = 'in' if inbound_begin_round <= fr <= inbound_end_round else 'out'
                    count_dof(fr, fb, where)
                    changed = True

        if not changed:
            break

    # 最后构造 annotated_path：将所有 active block 分组（如果两个 block 通过 XOR 直接相等关联，则给它们相同组 id）
    # 我们用并查集 (union-find) 来收集通过 "直接等于" 关系连接的块：
    parent = {}
    def find(x):
        if parent.setdefault(x,x) != x:
            parent[x] = find(parent[x])
        return parent[x]
    def union(a,b):
        ra, rb = find(a), find(b)
        if ra != rb:
            parent[rb] = ra

    # 若 dependencies 显示 (t,b) = XOR of sources, 那么可以把这些 source 相互 union（表示它们之间的差分有关联/等同）
    for (t,b), sources in dependencies.items():
        if len(sources) >= 2:
            # union all sources together and with target (t,b)
            nodes = [(s[0], s[1]) for s in sources]
            nodes.append((t,b))
            for i in range(len(nodes)-1):
                union(nodes[i], nodes[i+1])

    # 为所有 inactive block 分配组 id 0；active block 分配 union-find 的组 id 编号（紧凑化）
    group_map = {}
    next_gid = 1
    annotated_path = [[0]*B for _ in range(R)]
    for r in range(R):
        for b in range(B):
            if not active.get((r,b), False):
                annotated_path[r][b] = 0
            else:
                root = find((r,b))
                if root is None:
                    # isolated active, 给自己一个组
                    annotated_path[r][b] = next_gid
                    next_gid += 1
                else:
                    if root not in group_map:
                        group_map[root] = next_gid
                        next_gid += 1
                    annotated_path[r][b] = group_map[root]

    # 返回结果
    return {
        'DoF_In': DoF_In,
        'Prob_In_xor': Prob_In_xor,
        'Prob_In_MC': Prob_In_MC,
        'DoF_Out': DoF_Out,
        'Prob_Out_xor': Prob_Out_xor,
        'Prob_Out_MC': Prob_Out_MC,
        'annotated_path': annotated_path
    }


# --------------------------
# 执行检查并打印结果（示例）
# --------------------------

if __name__ == "__main__":
    res = propagate_and_count(
        solution, dependencies,
        inbound_begin_round, inbound_end_round,
        inbound_start_blocks, inbound_end_blocks,
        outbound_begin_round, outbound_end_round,
        outbound_start_blocks, outbound_end_blocks
    )

    print("=== 检查结果 ===")
    print("DoF_In (自由度计数，单位: 个活跃字节):", res['DoF_In'])
    print("Prob_In_xor:", res['Prob_In_xor'])
    print("Prob_In_MC:", res['Prob_In_MC'])
    print("DoF_Out:", res['DoF_Out'])
    print("Prob_Out_xor:", res['Prob_Out_xor'])
    print("Prob_Out_MC:", res['Prob_Out_MC'])
    print()
    print("Annotated path (group id matrix, 0=inactive):")
    for r in range(len(res['annotated_path'])):
        print("r%02d:" % r, res['annotated_path'][r])


=== 检查结果 ===
DoF_In (自由度计数，单位: 个活跃字节): 5
Prob_In_xor: 0
Prob_In_MC: 24960
DoF_Out: 10
Prob_Out_xor: 0
Prob_Out_MC: 24960

Annotated path (group id matrix, 0=inactive):
r00: [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
r01: [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
r02: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
r03: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
r04: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
r05: [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
r06: [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
r07: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
r08: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
r09: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [16]:
import json
import numpy as np

# 参数
rounds = 7
num_blocks = 12
block_size = 16
max_vars = rounds * num_blocks * block_size  # 7*12*16=1344

# 读取 JSON
with open('checkpoint_4.json', 'r') as f:
    solution_data = json.load(f)

# 初始化三维数组
solution_array = [[[0 for _ in range(block_size)] for _ in range(num_blocks)] for _ in range(rounds)]

count = 0
for key, val in solution_data.items():
    if count >= max_vars:
        break  # 只取前 7*12*16 个元素
    parts = key.split('_')  # 假设 key 格式是 x_r_b_j
    r, b, j = int(parts[1]), int(parts[2]), int(parts[3])
    if r < rounds and b < num_blocks and j < block_size:
        solution_array[r][b][j] = int(round(val))
        count += 1

S= solution_array
# S = [[] for _ in range(rounds)]
# for i in range(rounds):
#     for j in range(num_blocks):
#         S[i].append((solution_array[i][j]))
# print(S)

In [24]:
class ComputeFreedom:

    def __init__(self, S):
        self.__SBLocation = []
        self.__S = S

    def InboundSB_forward(self, r, b):#####################b是block的索引

        l = self.__S[r][b]
        for byte_index, val in enumerate(l):
            if val == 1:
                self.__SBLocation.append((r, b, byte_index))

    def get_SBLocation(self):
        """返回记录的活跃字节坐标"""
        return self.__SBLocation

In [30]:
cf = ComputeFreedom(S)
cf.InboundSB_forward(0, 5)
cf.InboundSB_forward(1, 8)

print(cf.get_SBLocation())

[(0, 5, 4), (0, 5, 6), (0, 5, 7), (0, 5, 8), (0, 5, 9), (0, 5, 11), (0, 5, 12), (0, 5, 13), (0, 5, 14)]


In [ ]:
class ActiveGroups:
    def __init__(self, S, rounds, num_blocks=12, block_size=16):
        """
        S: 截断差分路径 (rounds  num_blocks  block_size)
        rounds: 轮数
        num_blocks: 每轮的block数 (默认12)
        block_size: 每个block的字节数 (默认16)
        """
        self.S = S
        self.rounds = rounds*2 #####################SB两岸增加了一个state
        self.num_blocks = num_blocks
        self.block_size = block_size

        # group_id[r][b][byte] = 属于哪个组编号
        self.group_id = [[[None for _ in range(block_size)] 
                          for _ in range(num_blocks)] 
                         for _ in range(rounds)]
        
        self.current_group = 0   # 下一个可用组编号

    def new_group(self):
        """新建一个组编号"""
        self.current_group += 1
        g = self.current_group
        return g

    def assign_group(self, r, b, byte, group):
        """为某个字节赋组编号"""
        self.group_id[r][b][byte] = group

    def propagate_linear(self, r, b, byte, r_next, b_next, byte_next):
        """线性传播 (SR, MC, XOR 0)"""
        g = self.group_id[r][b][byte]
        if g is None:  # 如果当前还没分组#################这个if是否可以删掉
            g = self.new_group()
            self.assign_group(r, b, byte, g)
        self.assign_group(r_next, b_next, byte_next, g)

    def propagate_sb(self, r, b, byte, r_next, b_next, byte_next):
        """S-box 传播 (必须新起一组)"""
        g_new = self.new_group()
        self.assign_group(r_next, b_next, byte_next, g_new)

    # def propagate_xor(self, r, b, bytes_in, r_next, b_next, byte_next):
    #     """XOR 传播 (多个输入 → 新起一组)"""
    #     if self.S[r][b][bytes_in] == 1:
    #         g_new = self.new_group()
    #         self.assign_group(r_next, b_next, byte_next, g_new)

    def get_groups(self):
        return self.group_id

    def sb_block(self, r, b, r_next, b_next):##################经过SB后整个块同一成一类

        for byte in range(self.block_size):
            byte_next = byte
            if self.S[r][b][byte] == 1:
                self.propagate_sb(r, b, byte, r_next, b_next, byte_next)#########先找到第一个1，定义一个新类
                g = self.current_group
                break
        for byte_rest in range(byte+1, self.block_size):
            byte_next = byte_rest
            if self.S[r][b][byte_rest] == 1:
                self.assign_group(r_next, b_next, byte_next, g)#####后面都用g的族类
    
    def sb_block_non(self, r, b, r_next, b_next):
        for byte in range(self.block_size):
            byte_next = byte
            if self.S[r][b][byte] == 1:
                self.propagate_linear(r, b, byte, r_next, b_next, byte_next)

    def srmcak_block(self, r1, b1, r2,  b2, r_next, b_next):
        for byte_next in range(self.block_size):########注意这里没有考虑mc之后为0+后面一个1输出一个1，实在不知道怎么建模
            if self.S[r_next][b_next][byte_next] == 1:
                if self.S[r2][b2][byte_next] == 1:#############块2的字节索引和输出一样
                    self. propagate_sb(r2, b2, byte_next, r_next, b_next, byte_next)
                else:###################那就说明这里的活跃1是MC带来的
                    byte = self.S[r1][b1].index(1)#################不可能没有1，否则的话还输出1，只能说明是块2带来的
                    g = self.group_id[r1][b1][byte]#########刚过SB都是一个组
                    self.assign_group(r_next, b_next, byte_next, g)

    def xor_block(self, r1, b1, r2,  b2, r_next, b_next):
        for byte_next in range(self.block_size):
            if self.S[r_next][b_next][byte_next] == 1:
                if self.S[r1][b1][byte_next] == 1 and self.S[r2][b2][byte_next] == 1:
                    self. propagate_sb(r1, b1, byte_next, r_next, b_next, byte_next)
                elif self.S[r1][b1][byte_next] == 1:
                    self.propagate_linear(r1, b1, byte_next, r_next, b_next, byte_next)
                elif self.S[r2][b2][byte_next] == 1:
                    self.propagate_linear(r2, b2, byte_next, r_next, b_next, byte_next)

In [37]:
solution_array[0][0][12]
duplicated_solution = [s for state in solution_array for s in (state, state)]
for i, st in enumerate(duplicated_solution):
    print(f"state{i}:", st)

state0: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
state1: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [35]:
solution_array

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0,